# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [92]:
# Initial imports
import numpy as np
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Harrison\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [70]:
# Read your api key environment variable
load_dotenv("key.env")
api_key = os.getenv("NEWS_API")

In [71]:
# Create a newsapi client
#!pip install newsapi-python
from newsapi.newsapi_client import NewsApiClient
newsapi = NewsApiClient(api_key=api_key)

In [72]:
# Fetch the Bitcoin news articles
Bitcoin_news_en = newsapi.get_everything(
    q="Bitcoin",
    language="en",
    sort_by="relevancy"
)

In [73]:
# Fetch the Ethereum news articles
Ethereum_news_en = newsapi.get_everything(
    q="Ethereum",
    language="en",
    sort_by="relevancy"
)

In [74]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiments = []

for article in Bitcoin_news_en["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        bitcoin_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
bitcoin_df = pd.DataFrame(bitcoin_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
bitcoin_df = bitcoin_df[cols]

bitcoin_df.head()

,date,text,compound,positive,negative,neutral
0,2021-05-12,Just weeks after Tesla started accepting Bitco...,0.3818,0.071,0.00,0.929
1,2021-06-09,El Salvador's President Nayib Bukele has made ...,0.8402,0.282,0.00,0.718
2,2021-05-12,Image: Tesla\r\n\n \n\n Tesla has stopped acce...,0.4939,0.134,0.05,0.816
3,2021-06-09,El Salvador has become the first country in th...,0.1280,0.043,0.00,0.957
4,2021-05-19,Illustration by Alex Castro / The Verge\r\n\n ...,0.0000,0.000,0.00,1.000


In [75]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in Ethereum_news_en["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ethereum_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ethereum_df = pd.DataFrame(ethereum_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
ethereum_df = ethereum_df[cols]

ethereum_df.head()

,date,text,compound,positive,negative,neutral
0,2021-05-12,"Vitalik Buterin, the creator of Ethereum, on W...",0.2263,0.060,0.000,0.940
1,2021-05-15,Solana isn’t known yet outside of the crypto c...,0.4019,0.083,0.000,0.917
2,2021-05-19,"Bitcoin, Ethereum and a host of Altcoins suffe...",-0.2023,0.066,0.087,0.847
3,2021-05-20,,0.0000,0.000,0.000,0.000
4,2021-05-31,A representation of virtual currency Ethereum ...,0.0000,0.000,0.000,1.000


In [76]:
# Describe the Bitcoin Sentiment
bitcoin_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.000000,20.00000
mean,0.278520,0.097750,0.035500,0.86680
std,0.470441,0.079796,0.051382,0.07632
min,-0.762700,0.000000,0.000000,0.71800
25%,0.096000,0.055750,0.000000,0.81900
50%,0.381800,0.071000,0.000000,0.87600
75%,0.577875,0.155250,0.068000,0.92900
max,0.845500,0.282000,0.180000,1.00000


In [77]:
# Describe the Ethereum Sentiment
ethereum_df.describe()

,compound,positive,negative,neutral
count,20.000000,20.000000,20.00000,20.000000
mean,-0.011490,0.045300,0.04320,0.861500
std,0.373397,0.050086,0.07840,0.218677
min,-0.868900,0.000000,0.00000,0.000000
25%,-0.236725,0.000000,0.00000,0.838250
50%,0.000000,0.055500,0.00000,0.924500
75%,0.226300,0.068000,0.07125,0.961750
max,0.670500,0.188000,0.28600,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin Mean Positive Score : 0.097750 > Ethereum Mean Positive Score : 0.045300

Q: Which coin had the highest compound score?

A: Bitcoin Highest Compound score : 0.845500 > Ethereum Highest Compound Score : 0.670500

Q. Which coin had the highest positive score?

A: Bitcoin Highest Positive score : 0.282000 > Ethereum Highest Positive Score : 0.188000

---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [134]:
import string
!pip install nltk
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
!pip install re
#import re

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Harrison\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Harrison\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Harrison\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Harrison\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [135]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
sw = set(stopwords.words('english'))
first_result = [word.lower() for word in bitcoin_df["text"] if word.lower() not in sw]
# Expand the default stopwords list if necessary
first_result

['just weeks after tesla started accepting bitcoin as currency for cars, elon musk revealed in a tweet that it will "suspend" the effort. according to the release (tesla does not appear to have a funct… [+768 chars]',
 "el salvador's president nayib bukele has made good on his promise to adopt bitcoin as legal tender. officials in the central american country's congress voted to accept the cryptocurrency by a majori… [+1414 chars]",
 'image: tesla\r\n\n \n\n tesla has stopped accepting bitcoin as payment for its cars out of concern that it will contribute to greater consumption of fossil fuels, according to a statement ceo elon musk tw… [+853 chars]',
 'el salvador has become the first country in the world to recognize the cryptocurrency bitcoin as legal currency, according to president nayib bukele in a tweet on wednesday. citizens will be able to … [+3840 chars]',
 'illustration by alex castro / the verge\r\n\n \n\n cryptocurrency exchange coinbase is experiencing a “partial” outage 

In [138]:
# Complete the tokenizer function
def tokenizer_text(text):
    sw = set(stopwords.words('english'))
    regex = re.compile("[^a-zA-Z ]")
    re_clean = regex.sub('', article)
    words = word_tokenize(re_clean)
    lem = [lemmatizer.lemmatize(word) for word in words]
    output = [word.lower() for word in lem if word.lower() not in sw]
    return output

In [140]:
# Create a new tokens column for Bitcoin
BT = tokenizer(bitcoin_df["text"])
bitcoin_df["token"] = BT

TypeError: expected string or bytes-like object

In [141]:
# Create a new tokens column for Ethereum
ET = tokenizer(ethereum_df["text"])
ethereum_df_df["token"] = ET

NameError: name 'ethereum_df_df' is not defined

---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [15]:
from collections import Counter
from nltk import ngrams

In [16]:
# Generate the Bitcoin N-grams where N=2
bitcoin_counts = Counter(ngrams(BT, n=2))

In [17]:
# Generate the Ethereum N-grams where N=2
ethereum_counts = Counter(ngrams(ET, n=2))

In [18]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [19]:
# Use token_count to get the top 10 words for Bitcoin
token_count(BT)

In [20]:
# Use token_count to get the top 10 words for Ethereum
token_count(ET)

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [145]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

ModuleNotFoundError: No module named 'wordcloud'

In [146]:
# Generate the Bitcoin word cloud
bwc = WordCloud().generate(BT)
plt.imshow(bwc)

NameError: name 'WordCloud' is not defined

In [147]:
# Generate the Ethereum word cloud
ewc = WordCloud().generate(ET)
plt.imshow(ewc)

NameError: name 'WordCloud' is not defined

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [150]:
!pip install spacy
import spacy
from spacy import displacy

  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107097 sha256=7184671c0c1e6ab34bf82654d2e3144cc2367316abda772e28f492568ab99171
  Stored in directory: c:\users\harrison\appdata\local\pip\cache\wheels\11\73\9a\f91ac1f1816436b16423617c5be5db048697ff152a9c4346f2
Successfully built smart-open


In [151]:
# Download the language model for SpaCy
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2021-06-13 06:11:29.712349: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-06-13 06:11:29.712388: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [152]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

---
### Bitcoin NER

In [160]:
# Concatenate all of the Bitcoin text together
Bitcoin_Conc = ' '.join(bitcoin_df['text'])
Bitcoin_Conc

'Just weeks after Tesla started accepting Bitcoin as currency for cars, Elon Musk revealed in a tweet that it will "suspend" the effort. According to the release (Tesla does not appear to have a funct… [+768 chars] El Salvador\'s President Nayib Bukele has made good on his promise to adopt Bitcoin as legal tender. Officials in the Central American country\'s congress voted to accept the cryptocurrency by a majori… [+1414 chars] Image: Tesla\r\n\n \n\n Tesla has stopped accepting bitcoin as payment for its cars out of concern that it will contribute to greater consumption of fossil fuels, according to a statement CEO Elon Musk tw… [+853 chars] El Salvador has become the first country in the world to recognize the cryptocurrency bitcoin as legal currency, according to President Nayib Bukele in a tweet on Wednesday. Citizens will be able to … [+3840 chars] Illustration by Alex Castro / The Verge\r\n\n \n\n Cryptocurrency exchange Coinbase is experiencing a “partial” outage this morning fo

In [163]:
# Run the NER processor on all of the text
btc_doc = nlp(Bitcoin_Conc)

In [164]:
# Render the visualization
displacy.render(btc_doc, style='ent')

In [168]:
# List all Entities
print([ent.text for ent in btc_doc.ents])

['Just weeks', 'Tesla', 'Bitcoin', 'Elon Musk', "El Salvador's", 'Nayib Bukele', 'Central American', 'Tesla', 'Elon Musk', 'El Salvador', 'first', 'Nayib Bukele', 'Wednesday', 'Citizens', 'Alex Castro', 'The Verge\r\n\n \n\n Cryptocurrency', 'Coinbase', 'this morning', 'Coinbase', 'Bin', 'Last week', 'Musk', 'Dogecoin', 'Mark Zuckerberg', 'two', 'Max', 'Bitcoin', 'Elon Musk', 'March', 'Musk', 'Tesla', 'Earlier this year', 'EV', '1.5', 'Elon Musk', 'Bitcoin', 'Bitcoin', 'Altcoins', 'Tuesday night', 'Wednesday', 'morning', 'months', 'hundreds of billions', 'hours-long', 'this morning', 'May 19th', 'US', 'El Salvador', 'first', 'Bitcoin', 'Miami', 'Florida', 'last weekend', 'covid-19', 'Larry Cermak', 'Last week', 'Disney', 'This week', 'Tec', 'PayPals', 'Wednesday', 'Jose Fernandez da Pontethe', 'more than 5,500 miles', 'the United States', 'Mary-Ann RussonBusiness', 'BBC News', 'US', 'Donald Trump', 'Fox Business', 'Bitcoin', 'US', 'Photo', 'Michele Doying', 'Verge', 'Iran', 'Last week'

---

### Ethereum NER

In [169]:
# Concatenate all of the Ethereum text together
Ethereum_Conc = ' '.join(ethereum_df['text'])
Ethereum_Conc

'Vitalik Buterin, the creator of Ethereum, on Wednesday donated Ethereum and meme coins worth $1.5 billion in one of the largest-ever individual philanthropy efforts.\r\nButerin transferred 500 ETH and … [+1667 chars] Solana isn’t known yet outside of the crypto community. But insiders think the blockchain platform is interesting for a wide variety of reasons, beginning with its amiable founder, Anatoly Yakovenko,… [+7156 chars] Bitcoin, Ethereum and a host of Altcoins suffered massive drops Tuesday night and Wednesday morning, erasing months of gains and hundreds of billions in market cap. The overall crypto market shrunk m… [+1182 chars]  A representation of virtual currency Ethereum is seen in front of a stock graph in this illustration taken February 19, 2021. REUTERS/Dado Ruvic/Illustration/File PhotoCryptocurrency Ethereum extende… [+1099 chars] GPU shortages and inflated prices have become a byproduct of the growth of cryptomining. Needless to say, that\'s bad news for the gamer

In [170]:
# Run the NER processor on all of the text
eth_doc = nlp(Ethereum_Conc)

In [171]:
# Render the visualization
displacy.render(eth_doc, style='ent')

In [172]:
# List all Entities
print([ent.text for ent in eth_doc.ents])

['Vitalik Buterin', 'Ethereum', 'Wednesday', 'Ethereum', '$1.5 billion', 'Buterin', '500', 'ETH', 'Solana', 'Anatoly Yakovenko', 'Altcoins', 'Tuesday night', 'Wednesday', 'morning', 'months', 'hundreds of billions', 'Ethereum', 'February 19, 2021', 'GPU', 'Alex Castro', 'The Verge\r\n\n \n\n Cryptocurrency', 'Coinbase', 'this morning', 'Coinbase', 'Bin', 'Spanish', 'AI', 'Last May', 'Buterin', '27', '99.95%', 'Carl Beekhuizen', 'the Ethereum Foundation', 'Beekhuizen', 'tomorrow', 'Norton', 'Norton', 'Norton Crypto', 'Bitcoin', 'U.S. Dollar', 'May 26, 2020', 'Dado Ruvic/File PhotoBitcoin', 'last week', 'one-day', 'March last year', 'Wednesday', '$1 trillion', 'Wednesday', 'Entrepreneur', 'March 2021', 'more than one million', 'SafeMoon', 'about one hundred billion trillion dollars', 'decades', 'Ill', 'Jacks', 'CriddleTechnology', 'Kim Catdarshian', 'Ethereum', 'Spanish', 'AI', 'This week', 'March 2020']


---